In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import torch
from transformers import BertConfig
from fastformer import FastformerEncoder

c:\Users\madmo\miniconda3\envs\dl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
bert = BertModel.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

In [4]:
from data_utils import *
import pickle 

data_path = 'data/MINDsmall_train/'

build_news_id_map(data_path=data_path)
with open(data_path + 'news_id_map.pkl', 'rb') as f:
    news_id_map = pickle.load(f)
build_user_id_map(data_path=data_path)
with open(data_path + 'user_id_map.pkl', 'rb') as f:
    user_id_map = pickle.load(f)
build_news_token(news_id_map, data_path)
news_token = np.load(data_path + 'news_token.npy')

Tokenizing: 100%|██████████| 51283/51283 [01:10<00:00, 728.97it/s]


In [5]:
bert_features = precompute_bert_features(bert, news_token, 'cuda')
torch.save(bert_features, data_path+'bert_features.pt')

100%|██████████| 802/802 [00:31<00:00, 25.47it/s]


In [6]:
edge_index = build_edge_index(behavior_path=data_path+'behaviors.tsv', news_id_map=news_id_map, save_path=data_path+'edge_index.pt')

100%|██████████| 153727/153727 [00:04<00:00, 35247.40it/s]


In [7]:
build_behaviors(data_path + 'behaviors.tsv', user_id_map, news_id_map, data_path+'behaviors.pkl', 32)

156965it [00:03, 42245.42it/s]
